<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Custom embedddings con Gensim



### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. Se utilizará canciones de bandas para generar los embeddings, es decir, que los vectores tendrán la forma en función de como esa banda haya utilizado las palabras en sus canciones.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

### Datos
Utilizaremos como dataset el libro "The Adventures of Sherlock Holmes" de Arthur Conan Doyle

In [3]:
# Armar el dataset utilizando salto de línea para separar las oraciones/docs
df = pd.read_csv('the_adventures_of_sherlock_holmes.txt', sep='/n', engine='python', header=None)
df[100:105]

,0
100,"been burned, had you lived a few centuries ago..."
101,country walk on Thursday and came home in a dr...
102,have changed my clothes I can’t imagine how yo...
103,"Jane, she is incorrigible, and my wife has giv..."
104,"again, I fail to see how you work it out.”"


In [4]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 9626


### 1 - Preprocesamiento

In [5]:
from keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

In [6]:
# Demos un vistazo
sentence_tokens[50:52]

[['adler', 'of', 'dubious', 'and', 'questionable', 'memory'],
 ['i',
  'had',
  'seen',
  'little',
  'of',
  'holmes',
  'lately',
  'my',
  'marriage',
  'had',
  'drifted',
  'us',
  'away']]

### 2 - Crear los vectores (word2vec)

In [7]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [8]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                    window=2,       # cant de palabras antes y desp de la predicha
                    vector_size=400,       # dimensionalidad de los vectores 
                    negative=20,    # cantidad de negative samples... 0 es no se usa
                    workers=1,      # si tienen más cores pueden cambiar este valor
                    sg=1)           # modelo 0:CBOW  1:skipgram

In [9]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [10]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 9626


In [11]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.key_to_index))

Cantidad de words distintas en el corpus: 2231


### 3 - Entrenar el modelo generador

In [12]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                total_examples=w2v_model.corpus_count,
                epochs=20,
                compute_loss = True,
                callbacks=[callback()]
                )

Loss after epoch 0: 882543.625
Loss after epoch 1: 613979.25
Loss after epoch 2: 582623.625
Loss after epoch 3: 503463.75
Loss after epoch 4: 498194.0
Loss after epoch 5: 496698.0
Loss after epoch 6: 491409.25
Loss after epoch 7: 469579.5
Loss after epoch 8: 458565.5
Loss after epoch 9: 454541.0
Loss after epoch 10: 452995.0
Loss after epoch 11: 448325.0
Loss after epoch 12: 446506.0
Loss after epoch 13: 444282.5
Loss after epoch 14: 440983.0
Loss after epoch 15: 438874.0
Loss after epoch 16: 431306.0
Loss after epoch 17: 420712.0
Loss after epoch 18: 419144.0
Loss after epoch 19: 421645.0


(1358642, 2224020)

### 4 - Ensayar

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["watson"], topn=10)

[('doctor', 0.5544066429138184),
 ('scoundrel', 0.5528813004493713),
 ('fancies', 0.5386099219322205),
 ('prison', 0.5357534885406494),
 ('suppose', 0.5331242680549622),
 ('dad', 0.5330371260643005),
 ('“perhaps', 0.5211277604103088),
 ('madam', 0.5177298784255981),
 ('“ha', 0.5131778717041016),
 ('thief', 0.5125296711921692)]

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["police"], topn=10)

[('sell', 0.49815037846565247),
 ('station', 0.4929080307483673),
 ('official', 0.4887022376060486),
 ('successful', 0.46999576687812805),
 ('paddington', 0.4625307321548462),
 ('county', 0.4597383439540863),
 ('begin', 0.45728635787963867),
 ('master', 0.4513097405433655),
 ('names', 0.44859105348587036),
 ('cabman', 0.4462090730667114)]

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["sherlock"], topn=10)

[('adventures', 0.5049884915351868),
 ('description', 0.4986458718776703),
 ('jabez', 0.49620577692985535),
 ('“come', 0.49538201093673706),
 ('“holmes', 0.4941169023513794),
 ('“which', 0.49400094151496887),
 ('breckinridge', 0.49189236760139465),
 ('engineer', 0.4801810383796692),
 ('“my', 0.4769778549671173),
 ('godfrey', 0.4756881892681122)]

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["money"], topn=5)

[('service', 0.5363479256629944),
 ('£', 0.529644250869751),
 ('keeping', 0.5126955509185791),
 ('confidence', 0.5062758922576904),
 ('practice', 0.5047531723976135)]

### 5 - Visualizar agrupación de vectores

In [13]:
from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE                   
import numpy as np                                  

def reduce_dimensions(model):
    num_dimensions = 2  

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)  

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels

In [14]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

x_vals, y_vals, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=x_vals[:MAX_WORDS], y=y_vals[:MAX_WORDS], text=labels[:MAX_WORDS])
fig.show(renderer='colab') # esto para plotly en colab

### Alumno

- Se observaron algunos resultados interesantes al buscar palabras relacionadas:
* Watson -> doctor
* Police -> official, county, station
* Money -> service, £
* Sherlock -> Holmes, engineer, adventure
- Por otro lado, analizando el gráfico de los embeddings en 2D no están claras las relaciones o similitudes entre palabras. Posiblemente con mayor analisis exploratorio del corpus y mejorando el preprocesamiento se puedan mejorar los resultados.